## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-00-43-14 +0000,bbc,"Days after Texas floods, at least 161 people a...",https://www.bbc.com/news/articles/cql04vz0q45o
1,2025-07-09-00-39-32 +0000,nyt,Trump’s Frustration With Putin Preceded Resump...,https://www.nytimes.com/2025/07/08/us/politics...
2,2025-07-09-00-34-08 +0000,bbc,Are India's skies safe? Air safety watchdog re...,https://www.bbc.com/news/articles/cvgnr828z0go
3,2025-07-09-00-07-35 +0000,bbc,"US will hike tariffs on copper to 50%, Trump says",https://www.bbc.com/news/articles/cjrljr8qv3zo
4,2025-07-09-00-01-04 +0000,bbc,Trump and Netanyahu meet for a second time ami...,https://www.bbc.com/news/articles/ce3newwl1zeo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2302/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,25
72,new,6
26,tariffs,6
1,texas,5
213,minister,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...,46
39,2025-07-08-19-28-44 +0000,nypost,Zelensky’s right-hand man thanks Trump for rev...,https://nypost.com/2025/07/08/us-news/zelensky...,45
56,2025-07-08-16-43-22 +0000,nypost,Trump rips Putin during cabinet meeting for ta...,https://nypost.com/2025/07/08/us-news/trump-ri...,45
34,2025-07-08-19-57-30 +0000,nypost,Trump trolls ‘Palestinian senator’ Chuck Schum...,https://nypost.com/2025/07/08/us-news/trump-tr...,44
4,2025-07-09-00-01-04 +0000,bbc,Trump and Netanyahu meet for a second time ami...,https://www.bbc.com/news/articles/ce3newwl1zeo,42


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,46,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...
19,21,2025-07-08-22-42-03 +0000,nypost,Gavin Newsom says he’s ‘spellbound’ by AME bis...,https://nypost.com/2025/07/08/us-news/gavin-ne...
10,21,2025-07-08-23-16-20 +0000,nypost,Sen. Ted Cruz pictured at Parthenon on Greek v...,https://nypost.com/2025/07/08/us-news/ted-cruz...
39,20,2025-07-08-19-28-44 +0000,nypost,Zelensky’s right-hand man thanks Trump for rev...,https://nypost.com/2025/07/08/us-news/zelensky...
86,19,2025-07-08-09-29-38 +0000,bbc,Post Office report set to lay bare human impac...,https://www.bbc.com/news/articles/c4g267xe3y6o
7,18,2025-07-08-23-28-19 +0000,nypost,Powerful NYC teachers union endorses Zohran Ma...,https://nypost.com/2025/07/08/us-news/powerful...
8,18,2025-07-08-23-26-50 +0000,nypost,Mayor Eric Adams builds out re-election campai...,https://nypost.com/2025/07/08/us-news/mayor-er...
4,17,2025-07-09-00-01-04 +0000,bbc,Trump and Netanyahu meet for a second time ami...,https://www.bbc.com/news/articles/ce3newwl1zeo
42,17,2025-07-08-18-48-57 +0000,nypost,Trump bashes ‘idiot’ ex-Joint Chiefs Chairman ...,https://nypost.com/2025/07/08/us-news/donald-t...
47,16,2025-07-08-18-17-33 +0000,nypost,Pam Bondi deflects on whether Jeffrey Epstein ...,https://nypost.com/2025/07/08/us-news/donald-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
